# Extração da característica relação sinal-ruído (SNR) de dados de EEG

A relação sinal-ruído (SNR, do inglês _Signal-to-Noise Ratio_) é uma medida que compara o nível do sinal desejado com o nível do ruído de fundo. Uma SNR mais alta indica um sinal mais forte em comparação ao ruído, enquanto uma SNR mais baixa indica que o ruído é comparativamente forte.

O objetivo é utilizar um conjunto de dados com nível de atividade cerebral basal (que será considerado e utilizado como "ruído") e "removê-lo" do sinal obtido no experimento a fim de reconhecer os padrões de foco com maior clareza.

No contexto da caracterização de foco, podemos, a partir do sinal basal, classificar os sinais dos ritmos cerebrais com a presença ou não de foco, de forma que as amostras de sinais extraídas de um buffer sejam rotuladas com com a presença ou não de foco.

Esta análise será realizada com o auxílio do classificador SVM (_Support Vector Machine_). Nesse sentido, uma porcetagem das amostras são utilizadas para treino e outra para o teste (30 e 70% respectivamente).

In [55]:
import mne
import numpy as np
from scipy.signal import welch
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [56]:
# Transform a string that represents a time value ("min:sec") into number and converts it to seconds
def convert_min_to_sec(time):
    minutes, seconds = map(int, time.split(":"))
    return minutes * 60 + seconds

# Transfor a string thats represents a range time "0:21 - 0:40" into index that will be used to cut data
def convert_time_range_to_index(timerange):
    start, end = map(str, timerange.split(" - "))
    new_start = convert_min_to_sec(start)
    new_end = convert_min_to_sec(end)
    
    index = []
    index.append(new_start * 250)
    index.append(new_end * 250)

    return index

In [57]:
n_channels = 8
ch_types = ['eeg'] * n_channels
sfreq = 250
ch_names = ["F3", "Fz", "F4", "C3", "Cz", "C4", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_montage("standard_1020")

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [58]:
ia = '../dataset-s7/IA/OpenBCI-RAW-2023-09-28_16-51-25_IA.txt'
ia_ob = np.loadtxt(ia, delimiter=',', skiprows=5, usecols=range(1, 9))

basal = '../dataset-s7/TF/OpenBCI-RAW-2023-11-07_13-17-01_TF.txt'
basal_ob = np.loadtxt(basal, delimiter=',', skiprows=5, usecols=range(1, 9))

In [59]:
# Aula IA
ia_timeranges = [
    "6:00 - 10:17",
]

# Basal (TF)
basal_timerange = [
    "3:28 - 4:28"
]

In [60]:
ia_index = []
ia_index = convert_time_range_to_index(ia_timeranges[0])

basal_index = []
basal_index = convert_time_range_to_index(basal_timerange[0])

data_cut_ia = ia_ob[ia_index[0]:ia_index[1], :]
data_cut_basal = basal_ob[basal_index[0]:basal_index[1], :]

X = {
    'ia': mne.io.RawArray(data_cut_ia.T, info),
    # 'basal': mne.io.RawArray(data_cut_basal.T, info),
}

Creating RawArray with float64 data, n_channels=8, n_times=64250
    Range : 0 ... 64249 =      0.000 ...   256.996 secs
Ready.


### Filtragem dos dados

O laço a seguir aplica dois filtros nos dados. Primeiramente, atenua-se com um filtro _notch_ a frequência de 60Hz de forma a evitar a interferência da rede elétrica e, posteriormente, obtém-se a faixa de frequência desejada de 4 a 100Hz com um filtro passa faixa.

In [61]:
# TODO: filtrar corretamente os dados que são usados para SNR (basal)

for key in X:
    X[key].notch_filter(freqs=60)
    X[key].filter(l_freq=4, h_freq=100)

# X['ia'].notch_filter(freqs=60)
# X['ia'].filter(l_freq=4, h_freq=100)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband e

### Estimando o ruído de fundo através do sinal basal

In [62]:
# Lista que armazena as médias de potência para cada canal
noise_power = []

for channel_data in data_cut_basal.T:
    fft_result = np.fft.fft(channel_data)

    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    
    # média da potência no intervalo de tempo sem estímulo
    base_power = np.mean(psd)
    noise_power.append(base_power)

# média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
print(estimated_background_noise)

95920560085550.47


### Relação Sinal-Ruído de Banda Estreita

Banda estreita refere-se a sinais em que a largura de banda do sinal é pequena em relação à frequência central do sinal. Nesse sentido, a SNR de banda estreita é aplicada quando o sinal e o ruído ocupam uma faixa de frequência muito restrita. Ela pode ser observada pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

### Relação Sinal-Ruído de Banda Larga

Por outro lado, banda larga refere-se a sinais onde a largura de banda do sinal é grande, abrangendo uma ampla faixa de frequências. Dessa maneira, a SNR de banda larga aplica-se a situações nas quais tanto o sinal quanto o ruído ocupam uma larga faixa de frequências é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

### Aplicando as SNR de Banda Estreita e Larga para o sinal de interesse

In [63]:
target_amplitudes_adjusted = data_cut_ia - estimated_background_noise # data_cut_ia ou X['ia'] ?

narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
(64250, 8)
[[-57.10963119 -57.10963119 -57.10963119 ... -57.10963119 -57.10963119
  -57.10963119]
 [-57.10963119 -57.10963119 -57.10963119 ... -57.10963119 -57.10963119
  -57.10963119]
 [-57.10963119 -57.10963119 -57.10963119 ... -57.10963119 -57.10963119
  -57.10963119]
 ...
 [-57.10963119 -57.10963119 -57.10963119 ... -57.10963119 -57.10963119
  -57.10963119]
 [-57.10963119 -57.10963119 -57.10963119 ... -57.10963119 -57.10963119
  -57.10963119]
 [-57.10963119 -57.10963119 -57.10963119 ... -57.10963119 -57.10963119
  -57.10963119]]
(64250, 8)


/tmp/ipykernel_5531/1814287319.py:3: RuntimeWarning: invalid value encountered in log10
  narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)


### Janela Deslizante

In [64]:
sr = 250            # Taxa de amostragem dos dados do EEG (250Hz)
jump = 5            # Tamanho da janela de análise (5 em 5 segundos)
size = sr * jump    # Número de pontos de dados em cada janela de análise

# Definindo os limites das bandas de frequência (em Hz)
theta_band = (4, 8)       # Theta: 4 - 8 Hz
alpha_band = (8, 13)      # Alpha: 8 - 13 Hz
beta_band = (13, 30)      # Beta: 13 - 30 Hz
gamma_band = (30, 100)    # Gamma: 30 - 100 Hz


total_pc = {}
data_names = ('ia',)

# Mapeamento de índices para rótulos
band_labels = {
    0: 'Theta',
    2: 'Beta',
    3: 'Gamma'
}


for k, data in enumerate(X.values()):
    results = [0, 0, 0, 0] # Contagem de dominância de cada banda
    labels = [] # Lista para armazenar os rótulos das janelas

    data.set_eeg_reference(ref_channels='average', projection=False)
    
    # Percorre os dados em incrementos de sr (1 segundo) extraindo janelas de tamanho size (4s)
    for i in range(0, int(data.times[-1] - jump), 1):
        tmin = i
        tmax = i + jump

        cut = data.copy().crop(tmin=tmin, tmax=tmax).get_data()

        # Número de pontos por segmento
        n_per_seg = 128
        # Quantidade de sobreposição entre segmentos
        n_overlap = n_per_seg // 2 # 64
        
        # Calculando a densidade espectral de potência (PSD)
        freqs, psd = welch(cut, fs=sr, nperseg=n_per_seg, noverlap=n_overlap)
        
        # realizando a média dos 8 eletrodos
        X = np.average(psd, axis=0)

        # Encontrar os índices correspondentes às frequências de interesse
        theta_idxs = np.where((freqs >= theta_band[0]) & (freqs <= theta_band[1]))[0]
        alpha_idxs = np.where((freqs >= alpha_band[0]) & (freqs <= alpha_band[1]))[0]
        beta_idxs = np.where((freqs >= beta_band[0]) & (freqs <= beta_band[1]))[0]
        gamma_idxs = np.where((freqs >= gamma_band[0]) & (freqs <= gamma_band[1]))[0]

        # Calculando a potência em cada banda de frequência por meio da integração da PSD
        bands = [
            np.sum(X[theta_idxs]),
            np.sum(X[alpha_idxs]),
            np.sum(X[beta_idxs]),
            np.sum(X[gamma_idxs])
        ]
        
        # Determina qual banda é a dominante (maior potência)
        dominant_band = np.argmax(bands)
        
        # Calculando a pontuação de cada banda para mostrar a porcentagem
        results[dominant_band] += 1

        # Adiciona o rótulo correspondente, ou 'Não Focado' se não houver correspondência
        labels.append(band_labels.get(dominant_band, 'Não Focado'))

    # Armazenar os rótulos e porcentagens finais
    total_pc[data_names[k]] = {
        'Theta': labels.count('Theta'),
        'Beta': labels.count('Beta'),
        'Gamma': labels.count('Gamma'),
        'Não Focado': labels.count('Não Focado')
    }

    # total = sum(results)
    # percentages = [round((count/total) * 100, 2) for count in results]
    # print(data.__str__())
    # total_pc[data_names[k]] = percentages

print(total_pc)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


{'ia': {'Theta': 137, 'Beta': 0, 'Gamma': 114, 'Não Focado': 0}}


### Tarefa para aplicação das características SNR:

Agora que temos os dois vetores de características SNR, podemos utilizar buffers com e sem a evocação dos rítmos que caracterizam o foco.

#### Divisão dos dados

Utilizando a iteração (por exemplo, de 5 segundos caracterizada pela janela) realizada no sinal a cada ~1 segundo, realize a rotulação dos dados de interesse (Beta e Gamma). Ou seja, cada amostra será um sinal de 5 segundos (1250 pontos de 8 canais). A janela que não for qualificada como Beta ou Gama por exemplo, poderá ser rotulada com "desfoque". Se acharem interessante, adicionar rótulos do ritmo Theta também.

No caso do sinal que representa o basal (se tiverem) poderá pegar um único sinal de aproximadamente 30 segundos para ser utilizado na equação de ruído, que irá ter como resultado um único valor. Lembrando que o valor de ruído deve atuar no sinal no domínio da frequência.

#### Classificação

Cálculo: diferença entre tempo inicial (em minutos) e final (tbm em min) dada por: (tempo_min * 60) * 250. Aplicando aos dados reais para criar o vetor de características:

- Diferença: 90000 - 154250 com 8 canais
- Pontos totais: 64250 com 8 canais

- 64.250 (pontos totais) / 250 (taxa de amostragem) = 257 segundos
- 257 / 5 (tamanho da janela sem sobreposição) = 51 amostras (51,4)

<br>

Em nossos dados simulados, temos 150.000 pontos com 8 canais. A utilização desses dados funcionará da seguinte forma para a criação do vetor de características:

- 150.000 (pontos totais) / 250 (taxa de amostragem) = 600 segundos
- 600 / 5 (tamanho da janela sem sobreposição) = 120 amostras

| 1   | SNRw1                | SNRw2 | ... | SNRw8 | SNRn1 | SNRn2 | ... | SNRn8 |
|-----|----------------------|-------|-----|-------|-------|-------|-----|-------|
| 2   | [w1, w2, ..., w1250] |       |     |       |       |       |     |       |
| 3   |                      |       |     |       |       |       |     |       |
| ... |                      |       |     |       |       |       |     |       |
| 120 |                      |       |     |       |       |       |     |       |

- Agora transforme cada um dos vetores de pontos no domínio da frequência (1250 pontos) em um único valor real. Neste caso pode ser utilizado tanto a média como a mediana (ou ambos). Se utilizarmos as duas, teremos no final 32 colunas de características:
    - 8 canais
    - SNR narrow e SNR wide (2)
    - Média e mediana (2)

| 1   | 1   | ... | 32 |
|-----|-----|-----|----|
| 2   | w'  | ... |    |
| 3   | ... |     |    |
| ... |     |     |    |
| 120 |     |     |    |



Após obter o vetor de característica, realizar a divisão dos dados em treinamento e teste (normalmente uma proporção de 70 e 30% respectivamente) e aplicar para o classificador SVM.

**PLUS**: Ao final da tarefa, verificar a melhora dos resultados utilizando um seletor de características. Neste caso, podemos utilizar o RFE (*Recursive Feature Elimination*) em uma fase anterior a classificação para reduzir as 32 características se for necessário.

----

### Dúvidas

1 - `for channel_data in X['basal']: # precisa ser transposta? parece que ja ta transposta` <br>
2 - `target_amplitudes_adjusted = data_cut_ia - estimated_background_noise # data_cut_ia ou X['ia'] ?`

